<a href="https://colab.research.google.com/github/Clarxxon/JavaScript-Load-Image/blob/master/Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from scipy import integrate

In [9]:
# 1
def model1(x,w):
  # константы
  eps_p = 0.15
  eps_c = 0.08
  eps_m = 0.04
  z_0 = 0.98
  z_p = 0.98
  z_c = 0.98
  z_m = 1.03
  z_e = 1.2

  # задаем исходные величины
  w_12 = 1276
  Apl = [
       [1,0,0,0,0,0,0],
       [1,np.power((w-eps_p),2),np.power((w-eps_p),6),0,0,0,0],
       [1,np.power((w-eps_c),2),np.power((w-eps_c),6),0,0,0,0],
       [0,0,0,1,(w-eps_c),np.power((w-eps_c),2),np.power((w-eps_c),3)],
       [0,0,0,1,(w-eps_m),np.power((w-eps_m),2),np.power((w-eps_m),3)],
       [0,0,0,1,w,np.power(w,2),np.power(w,3)],
       [0,2*(w-eps_c),6*np.power((w-eps_c),5),0,-1,-2*(w-eps_c),-3*np.power((w-eps_c),2)]
  ]
  Bpl = [z_0,z_p,z_c,z_c,z_m,z_e,0]
  vW = np.matmul(np.linalg.inv(Apl),Bpl)
  f_c = vW[0]+vW[1]*np.power(x,2)+vW[2]*np.power(x,6)
  f_e = vW[3]+vW[4]*x+vW[5]*np.power(x,2)+vW[6]*np.power(x,3)

  def kxl(x,w):
    Kxl = 0 
    if (x>=0 and x<=(w-eps_c)):
      Kxl = f_c
    elif (x>=(w-eps_c)and x<=w):
      Kxl = f_e
    return Kxl
  
  Kx_x_w = kxl(x,w) if x >=0 else kxl(-1*x,w)

  W_c = w_12 * 0.001
  w_c = 0.5*W_c
  w = w_c
  Apl = [
        [1,0,0,0,0,0,0],
        [1,np.power((w-eps_p),2),np.power((w-eps_p),6),0,0,0,0],
        [1,np.power((w-eps_c),2),np.power((w-eps_c),6),0,0,0,0],
        [0,0,0,1,(w-eps_c),np.power((w-eps_c),2),np.power((w-eps_c),3)],
        [0,0,0,1,(w-eps_m),np.power((w-eps_m),2),np.power((w-eps_m),3)],
        [0,0,0,1,w,np.power(w,2),np.power(w,3)],
        [0,2*(w-eps_c),6*np.power((w-eps_c),5),0,-1,-2*(w-eps_c),-3*np.power((w-eps_c),2)]
  ]
  Bpl = [z_0,z_p,z_c,z_c,z_m,z_e,0]
  vW = np.matmul(np.linalg.inv(Apl),Bpl)

  x = abs(x)
  Kxl = 0 

  integral_res = 1
  if (x>=0 and x<=(w-eps_c)):
    def integrand(x):
      return  vW[0]+vW[1]*x**2+vW[2]*x**6
    integral_res = integrate.quad(integrand, -1*w_c, w_c)[0]

  elif (x>=(w-eps_c)and x<=w):
    def integrand(x):
      return  vW[3]+vW[4]*x+vW[5]*np.power(x,2)+vW[6]*np.power(x,3)
    integral_res = integrate.quad(integrand, -1*w_c, w_c)[0]

  K_pnorm = 1/W_c * integral_res
  K_px = Kx_x_w/K_pnorm

  return K_px

model1(5,12)



1.2504799987185458

In [36]:
# 2
input_params={
    S_x:0.005,
    L_bu:2,
    L_conic:0.93,
    Yu:1.8* 10**11,
    u:0.25,
    D_bu:1.6,
    L_ck:1.8,
    w_12:1276,
    p_12:40
  }

def model2(S_x=0.005,
    L_bu=2,
    L_conic=0.93,
    Yu=1.8* 10**11,
    u=0.25,
    D_bu=1.6,
    L_ck=1.8,
    w_12=1276,
    p_12=40): 

  aHC = L_bu + L_conic
  Ge = Yu/(2*(1+u))

  W_c = w_12*10**(-3)
  def Ybu1W(P):
    return P*W_c**2/(18.8*Yu*D_bu**4)*(12*aHC-8*W_c-6*L_ck-W_c**12/L_ck)

  def Ybu2W(P):
    return P*W_c/(Ge*3.14*D_bu**2)*(1-W_c/(2*L_ck))

  def YbuW(P):
    return 2*(Ybu1W(P)+Ybu2W(P))

  P_c = p_12*10**-4
  MpW=P_c/(YbuW(P_c))
  def yBU(z):
    return YbuW(P_c)*(2*z/W_c)**2

  def yBUd(j):
    return yBU(j*S_x)*10**6

  N_b = L_bu/2*S_x
  print(N_b)

  yBUd_j = [yBUd(j) for j in np.arange(0,N_b,S_x/60)]
  return yBUd_j

model2(*input_params)

0.005


[0.0,
 5.808781820443941e-21,
 2.3235127281775765e-20,
 5.2279036383995474e-20,
 9.294050912710306e-20,
 1.4521954551109854e-19,
 2.091161455359819e-19,
 2.846303092017531e-19,
 3.7176203650841224e-19,
 4.705113274559592e-19,
 5.808781820443942e-19,
 7.028626002737168e-19,
 8.364645821439276e-19,
 9.81684127655026e-19,
 1.1385212368070124e-18,
 1.3069759095998868e-18,
 1.487048146033649e-18,
 1.678737946108299e-18,
 1.8820453098238368e-18,
 2.0969702371802622e-18,
 2.3235127281775766e-18,
 2.5616727828157788e-18,
 2.811450401094867e-18,
 3.0728455830148453e-18,
 3.3458583285757104e-18,
 3.630488637777463e-18,
 3.926736510620104e-18,
 4.234601947103632e-18,
 4.5540849472280496e-18,
 4.885185510993355e-18,
 5.227903638399547e-18,
 5.582239329446627e-18,
 5.948192584134596e-18,
 6.325763402463451e-18,
 6.714951784433196e-18,
 7.115757730043827e-18,
 7.528181239295347e-18,
 7.952222312187756e-18,
 8.387880948721049e-18,
 8.835157148895231e-18,
 9.294050912710307e-18,
 9.764562240166265e-18

In [37]:
# 3

def model3(b1=0.7059,b2=-0.6714,b3=0.1946,z=1,sh=2):
  def uTc(x):
    return b1*x+b2*x**2+b3*x**3
  def uBc(x):
    return b1*x+b2*x**2+b3*x**3
  uGc0 = uTc(z+sh)+uBc(z-sh)
  return uGc0

model3()



-0.24259999999999993